In [151]:
import itertools
from itertools import groupby
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
import math

In [168]:
class DataProcessor(object):
    def __init__(self):
        self.client = MongoClient('localhost', port=27017)
        self.db = self.client.train
        self.train = self.db.Train
        self.station = self.db.Station 

        self.train_df = pd.DataFrame(self.train.find({}, {'_id': 0}))
        self.train_df = self.train_df.drop_duplicates(subset='No', keep='first', inplace=False)

        self.station_df = pd.DataFrame(self.station.find({}, {'_id': 0}))
        self.station_df = self.station_df.drop_duplicates(subset='name', keep='first', inplace=False)

        self.mergedTrainLs = [] 
        self.adjacency_df = []
                          
    def addTrainsOfStation(self):
        """
        将各个站点的经停列车加入Station表单
        :return: None
        """
        station = self.train_df.key.tolist()
        passby = list(zip(self.train_df.code, self.train_df.key))
        # 添加经过车站的列车
        trainsOfStation = {}
        for (code, names) in passby:
            for name in names:
                if name in trainsOfStation:
                    trainsOfStation[name].append(code)
                elif name != "":
                    trainsOfStation[name] = []
        for (name, codes) in trainsOfStation.items():
            #print(name, codes)              
            query = {"name":name}
            newvalues = { "$set": { "trains": codes } }
            self.station.update_one(query, newvalues)

    def generateMergedTrain(self):
        # 起止站作为第一特征，以距离作为第二特征量
        documentsDict = {}
        #counter = 0
        deleteLs = []
        for index,document in self.train_df.iterrows():
            start_s = document["start_s"]
            end_s = document["end_s"]
            km = document["km"]
            key1 = f"{start_s}-{end_s}-{km}"
            key2 = f"{end_s}-{start_s}-{km}"
            # counter += 1
            # if counter > 10:
            #     break
            # 起止有问题的不要
            if start_s == "" or end_s == "":
                #deleteLs.append(key1)
                continue
            # 分正向与反向两种情况讨论
            if key1 in documentsDict:
                documentsDict[key1]["forward"].append(document["code"])
            elif key2 in documentsDict:
                documentsDict[key2]["backward"].append(document["code"])
            else:
                documentsDict[key1] = {"start_s":start_s, "end_s":end_s, "type":document["type"],"distance":[], "time":[], "stations":[], "forward": [document["code"]], "backward":[]}

                prev = [0, 0, 0, 0, 0, 0, "0:00"]
                for item in document["info"]:
                    if item[1] == "": #筛去空站
                        continue
                    prev_t = prev[6].split(':')
                    prev_t = int(prev_t[0])*60+int(prev_t[1])
                    cur_t = item[6].split(':')
                    cur_t = int(cur_t[0])*60+int(cur_t[1])
                    # 出现时光倒流的不要
                    if cur_t < prev_t or int(item[5])-int(prev[5]) < 0:
                        documentsDict.pop(key1)
                        deleteLs.append(key1)
                        break
                    documentsDict[key1]["stations"].append(item[1])
                    documentsDict[key1]["distance"].append(int(item[5])-int(prev[5]))
                    documentsDict[key1]["time"].append(cur_t-prev_t)
                    prev = item
        self.mergedTrainLs = list(documentsDict.values())
        print("舍弃的列车:", len(deleteLs))
        print(deleteLs)


    # 构建邻接表
    def generateAdjacencyList(self):
        stationLs = self.station_df.name.tolist()
        #print(stationLs[:10])
        d = {col:pd.Series([[1e10, 1e10, 0]]*len(stationLs), index=stationLs) for col in stationLs}
        self.adjacency_df = pd.DataFrame(d)
        print("Finish initialization")
        #print(pd.DataFrame(d))
        for item in self.mergedTrainLs:
            #print()
            stations = item["stations"]
            pre_time = []  #时间前缀和
            pre_dis =[]    #距离前缀和
            pre = 0
            for it in item["time"]:
                pre_time.append(it+pre)
                pre = it+pre
            pre = 0
            for it in item["distance"]:
                pre_dis.append(it+pre)
                pre = it+pre
            # 更新任意两站间的关系
            for i in range(len(stations)):
                if not (stations[i] in stationLs):
                    continue
                for j in range(i+1, len(stations)):
                    if not (stations[j] in stationLs):
                        continue
                    ls = self.adjacency_df[stations[i]][stations[j]].copy()
                    # 更新时间依据
                    if ls[0] > pre_time[j]-pre_time[i]:
                        ls[0] = pre_time[j]-pre_time[i]
                        #ls[3] = item["type"]
                    # 更新空间依据
                    if ls[1] > pre_dis[j]-pre_dis[i]:
                        ls[1] = pre_dis[j]-pre_dis[i]
                        #ls[4] = item["type"]
                    ls[2] += (len(item["forward"])+len(item["backward"]))
                    self.adjacency_df[stations[i]][stations[j]] = ls.copy()
                    self.adjacency_df[stations[j]][stations[i]] = ls.copy()
            #print(item["start_s"]+item["end_s"]+"ok")
    
    # 计算可达性,3个最近的车站，直接相连的城市，使用经停列车数作为权重参数
    def calAccess(self):
        # 建立站-值对应表
        stationValue = {}
        for index,document in self.station_df.iterrows():
            if document["trains"] is np.nan: #nan是个大坑, 以及没有车经过的站居然存在
                stationValue[document["name"]] = -1
            elif len(document["trains"]) == 0:
                stationValue[document["name"]] = -1
            else:
                stationValue[document["name"]] = math.log(len(document["trains"]))

        # 计算每条路线带来的可达性
        trainAccess = {}   # 每条线的可达性
        passby = list(zip(self.train_df.code, self.train_df.key))
        for code, stations in passby:
            value = 0
            for station in stations:
                if station in stationValue:
                    value += stationValue[station]
            trainAccess[code] = value
        #print(trainAccess.items())
        
        # 计算每个站点的可达性
        stationAccess = {}
        for index,document in self.station_df.iterrows():
            if document["trains"] is np.nan: #nan是个大坑, 以及没有车经过的站居然存在
                stationAccess[document["name"]] = -1
            elif len(document["trains"]) == 0:
                stationAccess[document["name"]] = -1
            else:
                avalue = 0
                for code in document["trains"]:
                    if code in trainAccess:
                        avalue+= trainAccess[code]
                    # key = self.train_df[self.train_df["code"] == code].key
                    # if len(key) > 0:
                    #     key[0]
                stationAccess[document["name"]] = avalue

        # 构造可达性信息json文件
        with open("AccessInfo.json", 'w', encoding ="utf-8") as fw:
            fw.write(json.dumps(stationAccess, ensure_ascii=False))
        

# 实例化数据处理器
processor = DataProcessor()

In [169]:
processor.calAccess()

In [109]:
# 将同一线路的列车合并
processor.generateMergedTrain()

舍弃的列车: 32
['安达-齐齐哈尔南-1', '吐鲁番北-乌鲁木齐南-158', '吐鲁番北-乌鲁木齐南-158', '吐鲁番北-乌鲁木齐南-158', '深圳北-乐昌东-1499', '南京南-黄山北-739', '温州南-徐州东-1017', '北京-北安-1594', '昌图-铁岭-0', '莱西-枣庄西-0', '海拉尔-大庆-1090', '七台河-青山-606', '神树-白奎堡-321', '大连-沈阳-533', '八里甸子-小市-0', '八里甸子-小市-0', '淮北-水家湖-283', '北京-包头-697', '鸡西-富拉尔基-1157', '沈阳-元宝山-547', '凌源-小东-459', '赤峰南-小东-464', '包头-青岛北-1762', '武昌-十堰-493', '土地堂东-咸宁南-172', '西安-南宁-2211', '济南-伊宁-3795', '青岛-清河城-1753', '无锡-怀化-1941', '上海-伊宁-4742', '厦门北-南昌西-773', '哈尔滨西-天津西-1109']


In [ ]:
# !危险操作! 向数据库加入车站经停列车信息
processor.addTrainsOfStation()

In [94]:
# 构造新的列车信息json文件
with open("MergedTrainInfo.json", 'w', encoding ="utf-8") as fw:
    fw.write(json.dumps(processor.mergedTrainLs, ensure_ascii=False))

In [110]:
# 构建邻接矩阵
processor.generateAdjacencyList()

Finish initialization


In [95]:
print(processor.mergedTrainLs[2])

{'start_s': '沈阳北', 'end_s': '北京', 'type': '普通动车组', 'distance': [0, 20, 80, 39, 66, 263, 16, 148, 151], 'time': [0, 15, 38, 14, 26, 89, 20, 70, 73], 'stations': ['沈阳北', '沈阳南', '鞍山西', '海城西', '盘锦', '山海关', '秦皇岛', '唐山北', '北京'], 'forward': ['D14', 'D52', '2550', '4216'], 'backward': ['D51']}


In [111]:
print(processor.adjacency_df["北京南"]["济南东"])

[117, 410, 14]


In [112]:
# 储存邻接矩阵
processor.adjacency_df.to_csv("adjacency_table.csv")

In [117]:
# 读取邻接矩阵
ff= pd.read_csv("adjacency_table.csv", index_col=0)
print(ff["北京南"]["济南东"])

[117, 410, 14]


In [ ]:
# 计算可达性
